In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
import functools
import itertools
import pprint

import orbax.checkpoint
import numpy as np
import jax
import jax.numpy as jnp
import torch.utils.data.dataloader
import tensorflow as tf
import sqlalchemy as sa
import seaborn as sns
sns.set_theme(style='whitegrid', font_scale=1.3, palette=sns.color_palette('husl'),)
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from userdiffusion import samplers, unet
from userfm import cs, datasets, event_constraints, diffusion, sde_diffusion, flow_matching, utils, main as main_module, plots

2025-01-30 03:13:58.966942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738206838.994247   35023 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738206839.002662   35023 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/root/workspace/GitHub/pmlr-v202-finzi23a/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# somehow, this line of code prevents a segmentation fault in nn.Dense
# when calling model.init
tf.config.experimental.set_visible_devices([], 'GPU')

In [4]:
engine = cs.get_engine()
cs.create_all(engine)
session = cs.orm.Session(engine)
session.begin()

In [5]:
config_alt_ids = {
    # Lorenz
    # ('0y35hp7d', 'DM'): {},
    # ('3bjjfgwa', 'FM'): {'sample': {'use_score': True}},
    # ('c0ijllm1', 'FM+Reg'): {'sample': {'use_score': True}},
    # FitzHughNagumo
    ('jzke0dh6', 'DM'): {}, # jzke0dh6, epoch_1999, false, cs.DatasetFitzHughNagumo
    ('trauw532', 'FM'): {'sample': {'use_score': True}}, # trauw532, epoch_1999, false, cs.DatasetFitzHughNagumo
    ('7io88gsu', 'FM+Reg'): {'sample': {'use_score': True}}, # 7io88gsu, epoch_1999, false, cs.DatasetFitzHughNagumo
}

In [6]:
cfgs = session.execute(sa.select(cs.Config).where(cs.Config.alt_id.in_([c[0] for c in config_alt_ids])))
cfgs = {c.alt_id: c for (c,) in cfgs}
reference_cfg = cfgs[next(iter(cfgs.keys()))]
cfg_info = {}
for k in config_alt_ids:
    cfg = cfgs[k[0]]
    assert cfg.rng_seed == reference_cfg.rng_seed
    assert cfg.dataset == reference_cfg.dataset
    cfg_info[k] = dict(
        cfg=cfg,
    )

In [7]:
key = jax.random.key(reference_cfg.rng_seed)

In [8]:
key, key_dataset = jax.random.split(key)
reference_cfg.dataset.batch_count_test = 64
ds = datasets.get_dataset(reference_cfg.dataset, key=key_dataset)
splits = datasets.split_dataset(reference_cfg.dataset, ds)
dataloaders = {}
for n, s in splits.items():
    dataloaders[n] = torch.utils.data.dataloader.DataLoader(
        list(tf.data.Dataset.from_tensor_slices(s).batch(reference_cfg.dataset.batch_size).as_numpy_iterator()),
        batch_size=1,
        collate_fn=lambda x: x[0],
    )
data_std = splits['train'].std()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6450/6450 [21:00<00:00,  5.12it/s]


In [9]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
x_sample = next(iter(dataloaders['train']))
ckpt_name = 'epoch_1999'

for info in cfg_info.values():
    cfg = info['cfg']
    assert cfg.rng_seed == reference_cfg.rng_seed
    assert cfg.dataset == reference_cfg.dataset

    cfg_unet = unet.unet_64_config(
        splits['train'].shape[-1],
        base_channels=cfg.model.architecture.base_channel_count,
        attention=cfg.model.architecture.attention,
    )
    model = unet.UNet(cfg_unet)
    
    key, key_jaxlightning = jax.random.split(key)
    if isinstance(cfg.model, cs.ModelDiffusion):
        jax_lightning = diffusion.JaxLightning(cfg, key_jaxlightning, dataloaders, data_std, None, model)
    elif isinstance(cfg.model, cs.ModelFlowMatching):
        jax_lightning = flow_matching.JaxLightning(cfg, key_jaxlightning, dataloaders, data_std, None, model)
    else:
        raise ValueError(f'Unknown model: {cfg.model}')
        
    jax_lightning.params = orbax_checkpointer.restore(cfg.run_dir/ckpt_name)
    jax_lightning.params_ema = orbax_checkpointer.restore(cfg.run_dir/f'{ckpt_name}_ema')

    info['jax_lightning'] = jax_lightning

/root/workspace/GitHub/pmlr-v202-finzi23a/.venv/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1330: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [10]:
constraint = event_constraints.get_event_constraint(cfg.dataset)

In [11]:
nlls = defaultdict(lambda: defaultdict(list))
for (config_alt_id, source), info in cfg_info.items():
    # use same key for each model
    _, key_nll = jax.random.split(key)
    for batch in tqdm(dataloaders['test']):
        key_nll, key_nll_batch = jax.random.split(key_nll)
        if isinstance(info['cfg'].model, cs.ModelFlowMatching):
            x_noise, nll_no_div, nll = info['jax_lightning'].compute_nll(key_nll_batch, 1., batch, **config_alt_ids[k]['sample'])
        else:
            x_noise, nll_no_div, nll = info['jax_lightning'].compute_nll(key_nll_batch, 1., batch)
        nlls[(config_alt_id, source)]['nll_no_div'].append(nll_no_div)
        nlls[(config_alt_id, source)]['nll'].append(nll)

  0%|                                                                                                                                                                                                                                                          | 0/64 [00:00<?, ?it/s]

xf std 70.22528076171875 and std_max: 300.0


  3%|███████▌                                                                                                                                                                                                                                          | 2/64 [01:43<52:45, 51.06s/it]

xf std 74.07532501220703 and std_max: 300.0


  5%|███████████▎                                                                                                                                                                                                                                      | 3/64 [02:31<50:39, 49.83s/it]

...

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 63/64 [50:16<00:47, 47.60s/it]

xf std 82.04244995117188 and std_max: 300.0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [51:04<00:00, 47.88s/it]


xf std 80.90313720703125 and std_max: 300.0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [42:01<00:00, 39.40s/it]


In [12]:
nlls_concat = defaultdict(dict)
for k, out in nlls.items():
    for kk, arr in out.items():
        nlls_concat[k][kk] = jnp.concat(arr)
    print(f"{k=}, NLL={nlls_concat[k]['nll'].mean():.3f}")

k=('jzke0dh6', 'DM'), NLL=-7.365
k=('trauw532', 'FM'), NLL=-13.942
k=('7io88gsu', 'FM+Reg'), NLL=-14.408


In [15]:
# import pickle
# with open('nlls_fitzhugh.pkl', 'wb') as f:
#     pickle.dump(dict(jax.tree.map(np.array,nlls)), f)

In [16]:
# with open('nlls_fitzhugh.pkl', 'rb') as f:
#     test_nll = pickle.load(f)